# Data Loading and Cleaning

## Overview

Part of the frenzy that is doing these stats tends to create a bit of messy data collection code for a few reasons: the countdowns are "unannounced surprises", they have limited time spans, it's a side project, and I want to be timely.  Having said that, I wanted to clean it up a bit and codify what I've learned.  So here goes

## Setup

Under the covers it mostly a combo of [requests](http://docs.python-requests.org/en/master/) and [lxml](http://lxml.de/) for webscraping and (pandas)[https://pandas.pydata.org/] for data munging.  Before we get started, set up the imports

In [1]:
%matplotlib inline
from IPython.display import display, HTML
import requests 
from lxml import html
import pandas as pd
from datetime import date, datetime, time
from os import path, mkdir
import re


### Set up cache/data directories

In [2]:

cache_dir = './cache'
playlist_cache_dir = path.join(cache_dir, 'playlists')
a2z_cache_dir = path.join(cache_dir, 'a2z')
a2z70s_cache_dir = path.join(cache_dir, 'a2z70s')
data_dir = './data'

for d in (cache_dir, playlist_cache_dir, a2z_cache_dir, a2z70s_cache_dir, data_dir):
    if not path.exists(d): mkdir(d)

## Generic XPN Playlist scraping

Origionaly I tended to rely on the one-off countdown pages for playlists.
But eventually I ended up using the generic playlist at [http://xpn.org/playlists/xpn-playlist](http://xpn.org/playlists/xpn-playlist).
I've done this enough, it's past time to turn it into something reusable.

In [3]:
def fetch_playlist(day, cache_dir=None, verbose = False):
    """
    Fetches the XPN playlist for a given data
    
    Args:
        day (datetime.date) : The day to fetch the playlist for
        cache_dir (string)  : Path to the cache directory, or None to avoid caching
    
    Returns:
        DataFrame containing Artist and Title as Strings and Airtime as Timestamp
    """
    songs = pd.DataFrame(None, columns=['Artist', 'Title', 'Air Time'])
    if cache_dir is not None:
        cache_file =  path.join(cache_dir, "%04d-%02d-%02d.csv" % \
                                (day.year, day.month, day.day))
    if cache_file is not None and path.exists(cache_file):
        songs = pd.read_csv(cache_file)
        songs['Air Time'] = pd.to_datetime(songs['Air Time'], errors='coerce')
        if verbose: print "Got %d rows from %s" % (len(songs), cache_file)
    else:
        day_s = '%02d-%02d-%04d' % (day.month, day.day, day.year)
        page = requests.post('http://xpn.org/playlists/xpn-playlist',
                                 data = {'playlistdate': day_s})
        if verbose: print "fetching %s returned status %s" % (day_s, page.status_code)
        #tree = html.fromstring(page.content.replace('\x91', ' '))
        tree = html.fromstring(page.content)
        tracks = tree.xpath('//h3/a/text()')
        # not all rows are tracks, some are membership callouts
        # but real tracks start with times and are formatted
        # HH:MM [am|pm] Artist - Title
        # Note that I've seen titles with embedded dashes,
        # but so far no artist names with them.  This may be luck.
        # Special programs like World Cafe, Echos, ...
        # also start with an air time, but don't have useful track info
        # but those list the program inside bars
        # eg |World Cafe| -  "Wednesday 11-2-2016 Hour 2, Part 7"
        date_regex = re.compile("^\d{2}:\d{2}\s")
        line_count= 0
        track_count = 0
        for track in tracks:
            line_count += 1
            if date_regex.match(track) and track[9:10] != '|':
                (artist, title) = track[9:].split(' - ', 1)
                dt = datetime.strptime(track[:8], '%I:%M %p')
                air_time = datetime.combine(day, dt.time())
                if verbose: print "adding %s %s %s" % (artist, title, air_time)
                songs = songs.append({'Artist': artist,
                                      'Title': title,
                                      'Air Time': air_time},
                                     ignore_index = True)
                if verbose: print "size = %d" % len(songs)
                track_count += 1
            
        if verbose: print 'read %d line and added %d tracks' % (line_count, track_count)
        # Drop any duplicates, which are not uncommon
        songs = songs.drop_duplicates()
        if cache_file is not None:
            songs.to_csv(cache_file, index=False)
            if verbose: print 'write %d rows to %s' % (len(songs), cache_file)
    
    return songs


### XPN 70's A to Z playlist

The 70s's AtoZ started at 6:00 am on Nov 29 2107.
At this time it is still running.
The only interruptions have been for Free at Noons,
that don't appear in the playlist,
but show as gaps from 12 noon to 12:40 or so on Fridays.


In [4]:
start_time = datetime(2017, 11, 29, 06, 00)
end_time = datetime(2017,12,12, 19, 00)
seventies = pd.DataFrame(None, columns=['Artist', 'Title', 'Air Time'])
for day in pd.date_range(start_time.date(), end_time.date()):
    seventies = seventies.append(fetch_playlist(day, playlist_cache_dir), ignore_index=True)
# delete data before the start time
seventies = seventies[seventies['Air Time'] >= start_time]
# Cover what looks like a Free at Noon slid into the play list
seventies = seventies[seventies['Title'] != 'The Runner']
seventies = seventies.sort_values(by = 'Air Time')

print "got %d rows" % len(seventies)

got 4157 rows


Look at what we got a bit

In [5]:
HTML(seventies.head(5).to_html())

,Artist,Title,Air Time
219,Steeleye Span,A Calling-On Song,2017-11-29 06:02:00
218,Joni Mitchell,A Case Of You,2017-11-29 06:03:00
217,Boz Scaggs,A Clue,2017-11-29 06:07:00
216,Todd Rundgren,A Dream Goes On Forever,2017-11-29 06:13:00
215,Lou Reed,A Gift,2017-11-29 06:16:00


In [6]:
HTML(seventies.describe(include='all', percentiles=[]).to_html(na_rep=''))

,Artist,Title,Air Time
count,4157,4157,4157
unique,1028,4000,4154
top,David Bowie,She's Gone,2017-12-10 23:17:00
freq,63,3,2
first,,,2017-11-29 06:02:00
last,,,2017-12-12 18:54:00


### The Origional A-Z Playlist

In [7]:
start_time = datetime(2016, 11, 30, 06, 00)
end_time = datetime(2016, 12, 17, 13, 30)
originals = pd.DataFrame(None, columns=['Artist', 'Title', 'Air Time'])
for day in pd.date_range(start_time.date(), end_time.date()):
    originals = originals.append(fetch_playlist(day, playlist_cache_dir), ignore_index=True)
# delete data before the start time
originals = originals.drop_duplicates()
originals['Air Time'] = pd.to_datetime(originals['Air Time'], errors='coerce')
originals = originals[originals['Air Time'] >= start_time]
originals = originals[originals['Air Time'] < end_time]
originals = originals.sort_values(by = 'Air Time')

print "got %d rows" % len(originals)

got 5691 rows


## Augmenting the Data

### Scraping the Playlist Specific Pages

For the origional and 70s A-Z, but not the A-Z leftovers,
the station put up cpountdown specific pages with play lists
in a slightly different format.
One advantage of using them is that they only include tracks from the countdown,
avoiding any need for time checking the data.
Another is that for the 70s A-Z,
they added lists by year.
Given the pain it was to search MusicBrainz for songs and
figure out the year, that's worth having.


#### 70s A-Z Page


##### Alphabetical Lists
Now that I've moved to the main playlist,
I don't know that the alphabetical lists buy much.
Getting the first letter ourselves is pretty easy.
But since older versions of the code used it,
we'll at least archive them

In [8]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

seventies_by_letter = pd.DataFrame(None, columns = ['Title', 'Artist', 'Letter'])
for letter in alphabet:
    cache_file = path.join(a2z70s_cache_dir, '%s.csv' % letter)
    if path.exists(cache_file):
        df = pd.read_csv(cache_file)
    else:
        rows = []
        page = requests.get('http://xpn.org/static/az2017.php?q=%s' % letter)
        tree = html.fromstring(page.content)
        songs = tree.xpath('//li/text()')
        for song in songs:
            rows.append(song.rsplit(' - ', 1) + [letter])
        df = pd.DataFrame(rows, columns=['Title', 'Artist', 'Letter'])
    df.to_csv(cache_file, index=False)
    seventies_by_letter = seventies_by_letter.append(df, ignore_index=True)

print "got %d songs by letter" % len(seventies_by_letter)

got 4202 songs by letter


##### Lists by Year

In [9]:
years = map(str, range(1970,1980))
seventies_by_year = pd.DataFrame(None, columns = ['Title', 'Artist', 'Year'])
for year in years:
    cache_file = path.join(a2z70s_cache_dir, '%s.csv' % year)
    if path.exists(cache_file):
        df = pd.read_csv(cache_file)
    else:
        rows = []
        page = requests.get('http://xpn.org/static/az2017v2.php?q=%s' % year)
        tree = html.fromstring(page.content)
        songs = tree.xpath('//li/text()')
        for song in songs:
            rows.append(song.rsplit(' - ', 1) + [year])
        df = pd.DataFrame(rows, columns=['Title', 'Artist', 'Year'])
    df.to_csv(cache_file, index=False)
    seventies_by_year = seventies_by_year.append(df, ignore_index=True)

seventies_by_year.to_csv(path.join(data_dir, 'seventies_by_year.csv'))
print 'got %d songs by year' % len(seventies_by_year)

got 3699 songs by year


### Best and Worst

In [10]:
def fetch_best(playlist_url, pagecount):
    """
    Fetch data from the 885 best or 88 worst playlists.
    Both use the same format, just different urls and 
    more or fewer pages.
    
    Args:
        playlist_url (string) : base url for the playlist
        pagecount (int) : number of pages to ge
    Returns:
        DataFrame containing the track data
    """
    
    rows = []
    
    for page_no in range(1, pagecount + 1):
        args = {'page': page_no}
        page = requests.get(playlist_url, params = args)
        tree = html.fromstring(page.content)
        tracks = tree.xpath("//*/tr[@class='countdown']")
        for track in tracks:
            artist = track.xpath('./td[2]/text()')[0]
            title = track.xpath('./td[@class="song"]/text()')[0]
            rows.append([title, artist])
    df = pd.DataFrame(rows, columns = ['Title', 'Artist'])
    return df


In [11]:
best885_file = path.join(data_dir, '885best.csv')
if not path.exists(best885_file):
    best885 = fetch_best('http://www.xpn.org/music-artist/885-countdown/2014/885-countdown-2014',18)
    best885.to_csv(best885_file, index=False)
else:
    best885 = pd.read_csv(best885_file)
    
HTML(best885.head(5).to_html())
                     

,Title,Artist
0,Thunder Road,Bruce Springsteen
1,Like A Rolling Stone,Bob Dylan
2,Imagine,John Lennon
3,A Day In The Life,The Beatles
4,Born To Run,Bruce Springsteen


In [12]:
worst88_file = path.join(data_dir, '88worst.csv')
if not path.exists(worst88_file):
    worst88 = fetch_best('http://www.xpn.org/music-artist/885-countdown/2014/885-countdown-2014-88-worst',2)
    worst88.to_csv(worst88_file, index=False)
else:
    worst88 = pd.read_csv(worst88_file)
    
HTML(worst88.head(5).to_html())

,Title,Artist
0,We Built This City,Starship
1,Who Let The Dogs Out,Baha Men
2,Achy Breaky Heart,Billy Ray Cyrus
3,(You're) Having My Baby,Paul Anka
4,Macarena,Los Del Rio


### Putting it together

One might think that we can just join up the data.
However there is a catch.
There are some cases where one or more of the URLs will return legitimte duplicates.
For example two entries for the same song / artist at the same time in the main playlist page.
However there are also valid entries for the same song / artist,
at different times, released in different years.
The catch is that there is no common key between our three sources to join on.
If we dedupe on title and artist we drop real tracks.
But doing a cartesian product would generate 4 apparent tracks for two tracks.
So we need to build an artificial key.

In [13]:
seventies = seventies.sort_values(by='Air Time')
seventies['Play'] = pd.Series([0 for x in range(len(seventies.index))], index=seventies.index)
last = None
count = 0
for idx, row in seventies.iterrows():
    if last is None or last != (row['Title'], row['Artist']):
        last = (row['Title'], row['Artist'])
        count = 0
    else:
        count += 1
        seventies.loc[idx, 'Play'] = count

seventies_by_letter = seventies_by_letter.drop_duplicates()

seventies_by_year = seventies_by_year.sort_values(by=['Title', 'Artist'])
seventies_by_year['Play'] = pd.Series([0 for x in range(len(seventies_by_year.index))], index=seventies_by_year.index)
last = None
count = 0
for idx, row in seventies_by_year.iterrows():
    if last is None or last != (row['Title'], row['Artist']):
        last = (row['Title'], row['Artist'])
        count = 0
    else:
        count += 1
        seventies_by_year.loc[idx, 'Play'] = count

seventies = seventies.merge(seventies_by_year, how='left', on=['Artist', 'Title', 'Play'])
seventies = seventies.merge(seventies_by_letter, how='left', on=['Artist', 'Title'])
seventies['Year'] = seventies['Year'].fillna(0.0).astype(int)
seventies['Air Time'] = pd.to_datetime(seventies['Air Time'], errors='coerce')




### Extracting First Words

In [14]:
from nltk.tokenize import RegexpTokenizer
custom_tokenize = RegexpTokenizer("[\w'\-]+|[^\w'\s\-]").tokenize
seventies = seventies.join(seventies.apply(lambda x: custom_tokenize(x[1])[0], axis=1).to_frame('First Word'))


#### Adding First Letter to Origionals
For last year's playlist, we'll figure out the song letters the old fashioned way,
we'll just take the first element of the title.

In [15]:
originals = originals.join(originals.apply(lambda x: x[1][0].upper(), axis=1).to_frame('Letter'))

### Estimating Durations

Since we have air times, we can approximate durations by subtracting the air time from the next track's air times.  There are a couple catches with this
- we need to pass in an explicit end time for the last track, but that's minor
- we need to add some logic to 'skip over' the free at noons that happen on fridays form 12 noon till "like 12:40 or so" and don't appear in the playlist at all
- the granularity is a bit course, as it is on a one minute basis.  We could be off by almost two minutes per song, but it ought to even out.
- there's no clear way to account for "non-song time" like station promos, hosts introducing songs, station ids, and so forth.  Fortunatly, the percentage of time that is really music is pretty high thanks to XPN being listener supported.

We'll do this for both the current and origional playlists.

In [16]:
def estimate_durations(playlist, end_time=None):
    """
    Estimate the song durations
    Args: 
        playlist (DataFrame): playlist with minimally an 'Air Time' attribute
        end_time (datetime): end time of the play list, or None if still going
    Return:
        modified DataFrame with 'Duration' attribute added.
    """
    
    playlist['Duration'] = pd.Series([0 for x in range(len(playlist.index))], index=playlist.index)
    previous = None
    last_idx = None
    for idx, row in playlist.iterrows():
        if not previous is None:
            if row['Air Time'].date().weekday() == 4 and previous.hour == 11 and row['Air Time'].hour == 12:
                # We just fell into a free at noon
                playlist.loc[last_idx, 'Duration'] = 60 - previous.minute
            else:
                # just subtract this start from the previous
                delta = row['Air Time'] - previous
                playlist.loc[last_idx, 'Duration'] = delta.seconds / 60
        previous = row['Air Time']
        last_idx = idx

    # fixup the last row
    if end_time is not None:    
        delta = end_time - playlist.loc[last_idx,'Air Time']
        playlist.loc[last_idx, 'Duration'] = delta.seconds / 60
    
    return playlist
        

In [18]:
seventies = estimate_durations(seventies, datetime(2017,12,12, 19, 00))
orginals = estimate_durations(originals, datetime(2016, 12, 17, 13, 30 ))

## Checking the Results

In [19]:
HTML(seventies.head(10).to_html())

,Artist,Title,Air Time,Play,Year,Letter,First Word,Duration
0,Steeleye Span,A Calling-On Song,2017-11-29 06:02:00,0,1970,A,A,1
1,Joni Mitchell,A Case Of You,2017-11-29 06:03:00,0,1971,A,A,4
2,Boz Scaggs,A Clue,2017-11-29 06:07:00,0,1977,A,A,6
3,Todd Rundgren,A Dream Goes On Forever,2017-11-29 06:13:00,0,1974,A,A,3
4,Lou Reed,A Gift,2017-11-29 06:16:00,0,1975,A,A,7
5,Poco,A Good Feelin' To Know,2017-11-29 06:23:00,0,1972,A,A,3
6,Mac Davis,A Little Less Conversation,2017-11-29 06:26:00,0,1971,A,A,3
7,Neil Young,A Man Needs A Maid,2017-11-29 06:29:00,0,1972,A,A,4
8,Lou Rawls,A Natural Man,2017-11-29 06:33:00,0,1971,A,A,3
9,David Bowie,A New Career In A New Town,2017-11-29 06:36:00,0,1977,A,A,5


In [20]:
HTML(seventies.describe(include='all', percentiles=[]).to_html(na_rep=''))

,Artist,Title,Air Time,Play,Year,Letter,First Word,Duration
count,4157,4157,4157,4157.000000,4157.000000,4153,4157,4157.000000
unique,1028,4000,4154,,,26,1853,
top,David Bowie,She's Gone,2017-12-10 23:17:00,,,S,The,
freq,63,3,2,,,449,170,
first,,,2017-11-29 06:02:00,,,,,
last,,,2017-12-12 18:54:00,,,,,
mean,,,,0.003127,1751.709887,,,4.668270
std,,,,0.055841,624.754683,,,2.218651
min,,,,0.000000,0.000000,,,0.000000
50%,,,,0.000000,1974.000000,,,4.000000


In [21]:
HTML(originals.head(10).to_html())

,Artist,Title,Air Time,Letter,Duration
245,Jackson 5,ABC,2016-11-30 06:01:00,A,3
244,Elvis Presley,A Big Hunk O' Love,2016-11-30 06:04:00,A,2
243,Johnny Cash,A Boy Named Sue (live),2016-11-30 06:06:00,A,4
242,Joni Mitchell,A Case Of You,2016-11-30 06:10:00,A,6
241,Ernie K-Doe,A Certain Girl,2016-11-30 06:16:00,A,3
240,Warren Zevon,A Certain Girl,2016-11-30 06:19:00,A,5
239,Sheryl Crow,A Change,2016-11-30 06:24:00,A,4
238,Sam Cooke,A Change Is Gonna Come,2016-11-30 06:28:00,A,3
237,The Beatles,A Day In The Life,2016-11-30 06:31:00,A,5
236,Ray Barretto,A Deeper Shade Of Soul,2016-11-30 06:36:00,A,4


### Saving the data

In [22]:
seventies_data_file = path.join(data_dir, '70sA2Z.csv')
seventies.to_csv(seventies_data_file, index=False)
originals_data_file = path.join(data_dir, 'A2Z.csv')
originals.to_csv(originals_data_file, index=False)